In [1]:
%load_ext autoreload
%autoreload 2

from dacman_csv import *

In [2]:
from pathlib import Path

PATH_DATA = Path('data/test1/')
PATH_1 = PATH_DATA / '2019DacManBasicA.csv'
PATH_2 = PATH_DATA / '2019DacManBasicB.csv'

In [3]:
differ = TableDiffer(display_data=True)
differ

In [4]:
builder = differ.get_builder(PATH_1, PATH_2)
builder

displaying diff data:
old:


,Site,Location,Date,Temperature,Pressure,Misc
0,A,B,1/2/17,71,4,xkjdksfjsdlkkfjs
1,A,C,2/3/17,70,5,dsfdsfsdfdsf
2,B,DF,1/4/17,60,9,dfsdfwefefds
3,C,X,3/8/17,90,20,lijdfskljfs


new:


,Site,Location,Date,Temp,Pressure,Misc
0,A,B,1/2/17,71,78,xkjdksfjsdlkkfjs
1,A,CD,2/3/17,70,5,dsfdsfsdfdsf
2,B,DF,1/4/17,60,9,dfsdfwefefds
3,C,X,3/8/17,90,20,lijdfskljfs


In [5]:
builder.index_diff.to_record()

{'added': [],
 'deleted': [],
 'unchanged': [0, 1, 2, 3],
 'changed': [],
 '_context': {}}

In [6]:
builder.schema_diff.to_record()

{'added': ['Temp'],
 'deleted': ['Temperature'],
 'unchanged': ['Site', 'Location', 'Date', 'Pressure', 'Misc'],
 'changed': [],
 '_context': {}}

In [7]:
builder.build_records()

In [8]:
builder.records_builder.old

,Site,Location,Date,Pressure,Misc
0,A,B,1/2/17,4,xkjdksfjsdlkkfjs
1,A,C,2/3/17,5,dsfdsfsdfdsf
2,B,DF,1/4/17,9,dfsdfwefefds
3,C,X,3/8/17,20,lijdfskljfs


In [9]:
builder.records_builder.new

,Site,Location,Date,Pressure,Misc
0,A,B,1/2/17,78,xkjdksfjsdlkkfjs
1,A,CD,2/3/17,5,dsfdsfsdfdsf
2,B,DF,1/4/17,9,dfsdfwefefds
3,C,X,3/8/17,20,lijdfskljfs


In [10]:
builder.records_builder.changed

,row_id,column_id,old_value,new_value
0,0,Pressure,4,78
1,1,Location,C,CD


In [11]:
builder.records_diff

'<TableRecordsDiff>'

added

,Site,Location,Date,Pressure,Misc


deleted

,Site,Location,Date,Pressure,Misc


unchanged

,Site,Location,Date,Pressure,Misc
2,B,DF,1/4/17,9,dfsdfwefefds
3,C,X,3/8/17,20,lijdfskljfs


changed

,row_id,column_id,old_value,new_value
0,0,Pressure,4,78
1,1,Location,C,CD


In [12]:
table_diff = builder.get_diff(context=differ.params)
table_diff

report = table_diff.to_record()
report

{'schema': {'added': ['Temp'],
  'deleted': ['Temperature'],
  'unchanged': ['Site', 'Location', 'Date', 'Pressure', 'Misc'],
  'changed': [],
  '_context': {}},
 'index': {'added': [],
  'deleted': [],
  'unchanged': [0, 1, 2, 3],
  'changed': [],
  '_context': {}},
 'records': {'added': [],
  'deleted': [],
  'unchanged': [{'Site': 'B',
    'Location': 'DF',
    'Date': '1/4/17',
    'Pressure': 9,
    'Misc': 'dfsdfwefefds'},
   {'Site': 'C',
    'Location': 'X',
    'Date': '3/8/17',
    'Pressure': 20,
    'Misc': 'lijdfskljfs'}],
  'changed': [{'row_id': 0,
    'column_id': 'Pressure',
    'old_value': 4,
    'new_value': 78},
   {'row_id': 1,
    'column_id': 'Location',
    'old_value': 'C',
    'new_value': 'CD'}],
  '_context': {'comparison_fields': ['Site',
    'Location',
    'Date',
    'Pressure',
    'Misc']}},
 '_context': {'read_options': [{}, {}], 'index_columns': None}}

In [13]:
report['records']['_context']

{'comparison_fields': ['Site', 'Location', 'Date', 'Pressure', 'Misc']}

In [14]:
report['_context']

{'read_options': [{}, {}], 'index_columns': None}

It's important to emphasize that all this internal substructure is an implementation detail, mostly to group together certain functionality and keep the components small. It should be completely hidden to the differ's clients:

In [15]:
diff = TableDiffer()(PATH_1, PATH_2)
diff.to_record()

{'schema': {'added': ['Temp'],
  'deleted': ['Temperature'],
  'unchanged': ['Site', 'Location', 'Date', 'Pressure', 'Misc'],
  'changed': [],
  '_context': {}},
 'index': {'added': [],
  'deleted': [],
  'unchanged': [0, 1, 2, 3],
  'changed': [],
  '_context': {}},
 'records': {'added': [],
  'deleted': [],
  'unchanged': [{'Site': 'B',
    'Location': 'DF',
    'Date': '1/4/17',
    'Pressure': 9,
    'Misc': 'dfsdfwefefds'},
   {'Site': 'C',
    'Location': 'X',
    'Date': '3/8/17',
    'Pressure': 20,
    'Misc': 'lijdfskljfs'}],
  'changed': [{'row_id': 0,
    'column_id': 'Pressure',
    'old_value': 4,
    'new_value': 78},
   {'row_id': 1,
    'column_id': 'Location',
    'old_value': 'C',
    'new_value': 'CD'}],
  '_context': {'comparison_fields': ['Site',
    'Location',
    'Date',
    'Pressure',
    'Misc']}},
 '_context': {'old': 'data/test1/2019DacManBasicA.csv',
  'new': 'data/test1/2019DacManBasicB.csv',
  'diff_params': {'read_options': [{}, {}], 'index_columns': N

This `changed` unit of the `records` component is preliminary. I'm working on a column-oriented differ (comparing data columns, i.e. pandas `Series`/Numpy `array`s), which should provide high performance and also allow an easy way to implement user-facing functionality such as setting per-column diff criteria and tolerance/thresholds values.

In [16]:
diff.records.changed

,row_id,column_id,old_value,new_value
0,0,Pressure,4,78
1,1,Location,C,CD
